In [ ]:
#Importing packages # Make sure that you atcivaate correct environment conda activate scanpy
import pandas as pd
import scanpy as sc
import anndata as ad
#import louvain
#import leidenalg

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor="white")

#writing output file
results_file = "./heart_immune_scanpy.h5ad"

In [ ]:
adata = sc.read_h5ad('/Users/srivalli/Desktop/Heart/hca_heart_immune_download.h5ad')
adata.var_names_make_unique()
adata

In [ ]:
#MAKING FILES READY

#Reading h5ad data
h5data = ad.read_h5ad("/Users/srivalli/Desktop/Heart/hca_heart_immune_download.h5ad")

#Data structure of anndata file i.e., summary stastics of the data
ad.AnnData(h5data)
adata = ad.AnnData(h5data)

#making matrix with the anndata
matrix = adata.X

#Making var unique i.e., genes
adata.var_names_make_unique()


In [ ]:
#PREPROCESSING

#Data normalization
sc.pl.highest_expr_genes(adata, n_top=20)

#Filtering genes and cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
# annotate the group of mitochondrial genes as "mt" -- GENES OF INTEREST
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)

#Graphical representation of data

#Violin plots
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

#Scatter plots
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

In [ ]:
# If we want to slice/Filter the data
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :].copy()

#Normalizing data matrix to have 10,000 reads per cell
sc.pp.normalize_total(adata, target_sum=1e4)

#Logarithmic calculation
sc.pp.log1p(adata)

In [ ]:
#Identifying high variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

In [ ]:
#Saving data as a backup so we wont loos e it later
#Saving normalized and logarithmic data

adata.raw = adata

In [ ]:
#Scaling data
#Filtering
adata = adata[:, adata.var.highly_variable]

#Scaling to unit variance
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

#Scaling values having standard deviation gerater than 10
sc.pp.scale(adata, max_value=10)

In [ ]:
##PRINCIPAL COMPONENT ANALYSIS

#Reducing dimensions
sc.tl.pca(adata, svd_solver="arpack")

#Scatter plot for PCA componentsa

sc.pl.pca(adata, color= "CST3")

In [ ]:
#Number of PCs to be considered
sc.tl.tsne(adata)
adata

In [ ]:
#Estimates of PCA to be considered
sc.pl.pca_variance_ratio(adata, log=True)

#Saving results
adata.write(results_file)
adata

In [ ]:
#COMPUTING NEIGHBOUIRHOOD GRAPH
sc.pp.neighbors(adata)

In [ ]:
#EMBERDDING THE NEIGHBOURHODD GRAPH
sc.tl.louvain(adata)
sc.tl.paga(adata)
sc.tl.umap(adata)
sc.tl.umap(adata)

#Giving colour codes for better visulauization
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"])

#For raw data which is logarithmized and nortmalizedd 
sc.pl.umap(adata, color=["CST3", "NKG7", "PPBP"], use_raw=False)

In [ ]:
#Clustering the neighborhood graph
#Recommendded method Leiden graph-clustering method
sc.tl.leiden(
    adata,
    resolution=0.9,
    random_state=0,
    n_iterations=2,
    directed=False,
)

sc.pl.umap(adata, color=["leiden"])

In [ ]:
#Saving file
adata.write(results_file)

In [ ]:
#FINDING MARKER GENES

#Using t-test
sc.tl.rank_genes_groups(adata, "leiden", method="t-test")
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
#Using wilcoxon method

sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

adata.write(results_file)

In [ ]:
#Using logistic regression
sc.tl.rank_genes_groups(adata, "leiden", method="logreg", max_iter=1000)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
##Marking genes for later use

marker_genes = [
    *["IL7R", "CD79A", "MS4A1", "CD8A", "CD8B", "LYZ", "CD14"],
    *["LGALS3", "S100A8", "GNLY", "NKG7", "KLRB1"],
    *["FCGR3A", "MS4A7", "FCER1A", "CST3", "PPBP"],
]

In [ ]:
#reloading  data and looking at top genes in dataframe

adata = sc.read(results_file)
adata
pd.DataFrame(adata.uns["rank_genes_groups"]["names"]).head(5)